# 04 Inference for Time Series (HAC/Newey-West)

Why naive SE break; HAC SE and interpretation.


## Table of Contents
- [Assumptions](#assumptions)
- [Autocorrelation](#autocorrelation)
- [HAC SE](#hac-se)
- [Checkpoint (Self-Check)](#checkpoint-self-check)
- [Solutions (Reference)](#solutions-reference)


## Why This Notebook Matters
Regression is the bridge between statistics and ML. You will learn:
- single-factor vs multi-factor interpretation,
- robust standard errors,
- coefficient stability and multicollinearity.


## What You Will Produce
- (no file output; learning/analysis notebook)

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Treating coefficients as causal without a causal design.
- Ignoring multicollinearity (unstable coefficients).

## Matching Guide
- `docs/guides/02_regression/04_inference_time_series_hac.md`



## How To Use This Notebook
- This notebook is hands-on. Most code cells are incomplete on purpose.
- Complete each TODO, then run the cell.
- Use the matching guide (`docs/guides/02_regression/04_inference_time_series_hac.md`) for deep explanations and alternative examples.
- Write short interpretation notes as you go (what changed, why it matters).



<a id="environment-bootstrap"></a>
## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.



In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Learn why naive OLS standard errors often break in macro time series, and how HAC/Newey-West helps.

Big idea:
- coefficients answer "what best fits the line"
- standard errors answer "how uncertain are we about the coefficients"

Time series often violate the assumptions behind naive SE.



## Primer: statsmodels vs scikit-learn (Inference vs Prediction)

### Two different goals
- **Prediction (ML):** optimize out-of-sample accuracy.
- **Inference (econometrics):** interpret coefficients + quantify uncertainty.

scikit-learn is mostly prediction-focused.
statsmodels is built for inference (standard errors, p-values, confidence intervals).

### Minimal statsmodels OLS pattern
```python
import statsmodels.api as sm

# X: DataFrame of features
# y: Series target

# Add intercept column
# Xc = sm.add_constant(X, has_constant='add')

# Fit OLS
# res = sm.OLS(y, Xc).fit()
# print(res.summary())
```

### What you're looking at in `res.summary()`
Typical regression output includes:
- **coef**: estimated coefficients $\hat\beta$
- **std err**: estimated standard errors $\widehat{SE}(\hat\beta)$
- **t**: t-statistic $\hat\beta / \widehat{SE}(\hat\beta)$
- **P>|t|**: p-value for $H_0: \beta=0$ (under assumptions)
- **[0.025, 0.975]**: 95% confidence interval for $\beta$

You can access these programmatically:

```python
# res.params        # coefficients (pandas Series)
# res.bse           # standard errors
# res.pvalues       # p-values
# res.conf_int()    # confidence intervals
```

### Robust standard errors
Robust SE change uncertainty estimates, not coefficients.

```python
# HC3 (cross-section)
# res_hc3 = res.get_robustcov_results(cov_type='HC3')

# HAC/Newey-West (time series)
# res_hac = res.get_robustcov_results(cov_type='HAC', cov_kwds={'maxlags': 4})
```

Why this matters:
- Cross-sectional data often has **heteroskedasticity** (error variance changes across observations).
- Time series often has **autocorrelation** (errors correlated over time).

### Practical rule
- Use sklearn for predictive pipelines and cross-validation.
- Use statsmodels when you need inference and careful interpretation.


## Primer: Hypothesis Testing (p-values, t-tests, and Confidence Intervals)

You will see p-values, t-statistics, and confidence intervals in regression output (especially `statsmodels`).
This primer gives you the minimum you need to avoid the most common misunderstandings.

### Definitions (in plain language)
- **Hypothesis**: a claim about an unknown population quantity (a parameter).
- **Null hypothesis** $H_0$: the "default" claim (often “no effect”).
- **Alternative hypothesis** $H_1$: the claim you consider if the data looks inconsistent with $H_0$.
- **Test statistic**: a number computed from data that measures how surprising the data is under $H_0$.
- **p-value**: the probability (under the assumptions of the null model) of seeing a test statistic at least as extreme as you observed.
- **Significance level** $\alpha$: a pre-chosen cutoff (commonly 0.05) used to decide whether to reject $H_0$.

### What a p-value is NOT
- It is **not** the probability $H_0$ is true.
- It is **not** the probability your model is correct.
- It is **not** a measure of economic importance.

### Type I / Type II errors and power
- **Type I error (false positive)**: rejecting $H_0$ when $H_0$ is true. Rough probability $\approx \alpha$ under assumptions.
- **Type II error (false negative)**: failing to reject $H_0$ when $H_1$ is true.
- **Power**: $1 - P(\text{Type II error})$. Power increases with larger samples and larger true effects.

### Regression t-test intuition
In OLS regression we estimate coefficients $\hat\beta$. A common test is:
- $H_0: \beta_j = 0$ (no linear association between $x_j$ and $y$ holding other features fixed)

The **t-statistic** is:

$$
t_j = \frac{\hat\beta_j - 0}{\widehat{SE}(\hat\beta_j)}
$$

Interpretation (roughly):
- if $|t_j|$ is large, $\hat\beta_j$ is far from 0 relative to its uncertainty estimate
- if $|t_j|$ is small, the data is compatible with $\beta_j$ being near 0 (given assumptions)

### Confidence intervals (CI) connect to hypothesis tests
A 95% CI is usually reported as:

$$
\hat\beta_j \pm t_{0.975}\cdot \widehat{SE}(\hat\beta_j)
$$

If the 95% CI does not include 0, the two-sided p-value is typically < 0.05.

### Python demo (toy): one-sample t-test and a regression coefficient test
This is not your project data; it is purely to make the objects concrete.

```python
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy import stats

rng = np.random.default_rng(0)

# 1) One-sample t-test: is the mean of x equal to 0?
x = rng.normal(loc=0.2, scale=1.0, size=200)
t_stat, p_val = stats.ttest_1samp(x, popmean=0.0)
print("t-test t:", t_stat, "p:", p_val)

# 2) Regression t-test: is the slope on x equal to 0?
n = 300
x2 = rng.normal(size=n)
eps = rng.normal(scale=1.0, size=n)
y = 1.0 + 0.5 * x2 + eps

df = pd.DataFrame({"y": y, "x": x2})
X = sm.add_constant(df[["x"]])
res = sm.OLS(df["y"], X).fit()

# The summary includes coef, SE, t, p, and CI
print(res.summary())

# Manual t-stat for slope (matches summary output)
beta_hat = res.params["x"]
se_hat = res.bse["x"]
print("manual t:", beta_hat / se_hat)
```

### Common ways hypothesis testing goes wrong in ML + macro
- **Multiple testing**: you try many features/specifications; some will look “significant” by chance.
- **Violating assumptions**: autocorrelation and heteroskedasticity can make naive SE too small.
- **Confusing predictive success with causal claims**: a coefficient can predict well without being causal.

Practical guidance for this project:
- Report effect sizes + uncertainty, not just “significant / not significant.”
- For macro time series, prefer robust SE (HAC/Newey-West) when interpreting p-values.
- For predictive tasks, always complement p-values with out-of-sample evaluation.


<a id="assumptions"></a>
## Assumptions

### Goal
Fit a baseline regression and inspect residuals.

Reminder:
- OLS coefficients can be computed even when assumptions fail.
- Inference (SE / p-values / CI) is what becomes unreliable.



### Your Turn (1): Load macro data and pick a simple regression


In [ ]:
import pandas as pd

path = PROCESSED_DIR / 'macro_quarterly.csv'
if path.exists():
    df = pd.read_csv(path, index_col=0, parse_dates=True)
else:
    df = pd.read_csv(SAMPLE_DIR / 'macro_quarterly_sample.csv', index_col=0, parse_dates=True)

y_col = 'gdp_growth_qoq'
x_cols = ['T10Y2Y_lag1']

df_m = df[[y_col] + x_cols].dropna().copy()
df_m.tail()



### Your Turn (2): Fit OLS and compute residuals


In [ ]:
import statsmodels.api as sm

X = sm.add_constant(df_m[x_cols], has_constant='add')
y = df_m[y_col]
res = sm.OLS(y, X).fit()

# Residuals = y - y_hat
resid = res.resid
fitted = res.fittedvalues

print(res.summary())
resid.head()



### Your Turn (3): Residual diagnostics plots (simple but useful)


In [ ]:
import matplotlib.pyplot as plt

# TODO: Plot residuals over time
# TODO: Plot residuals vs fitted values (heteroskedasticity visual check)
...



<a id="autocorrelation"></a>
## Autocorrelation

### Goal
Measure whether residuals are correlated over time.

If residuals are autocorrelated, naive SE can be too small.



### Your Turn (1): Autocorrelation by lag


In [ ]:
# TODO: Compute residual autocorrelation at lags 1..8.
# Hint: resid.autocorr(lag=k)
ac = {k: float(resid.autocorr(lag=k)) for k in range(1, 9)}
ac



### Your Turn (2): A simple ACF plot


In [ ]:
import matplotlib.pyplot as plt

# TODO: Plot the autocorrelation values as a bar chart.
...



<a id="hac-se"></a>
## HAC SE

### Goal
Fit the same model but compute HAC/Newey-West robust SE.

Your task:
- compare coefficient (should be the same)
- compare SE/p-values (can differ)



### Your Turn (1): Compare naive vs HAC with different maxlags


In [ ]:
from src import econometrics

res_naive = econometrics.fit_ols(df_m, y_col=y_col, x_cols=x_cols)
res_hac1 = econometrics.fit_ols_hac(df_m, y_col=y_col, x_cols=x_cols, maxlags=1)
res_hac4 = econometrics.fit_ols_hac(df_m, y_col=y_col, x_cols=x_cols, maxlags=4)

# TODO: Print SE/p-values side-by-side for the slope coefficient.
...



### Your Turn (2): Write a careful interpretation


In [ ]:
# TODO: Write 6-10 sentences answering:
# - Did HAC increase or decrease your SE?
# - How did that affect your p-value/CI?
# - What assumptions are still required even with HAC?
notes = """
...
"""
print(notes)



<a id="checkpoint-self-check"></a>
## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
# TODO: After you build X/y and split by time, validate the split.
# Example (adjust variable names):
# assert X_train.index.max() < X_test.index.min()
# assert y_train.index.equals(X_train.index)
# assert y_test.index.equals(X_test.index)
# assert not X_train.isna().any().any()
# assert not X_test.isna().any().any()
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



<a id="solutions-reference"></a>
## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Assumptions</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 04_inference_time_series_hac — Assumptions
# OLS assumptions (esp. independent errors) often fail in time series.
```

</details>

<details><summary>Solution: Autocorrelation</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 04_inference_time_series_hac — Autocorrelation
import pandas as pd
import numpy as np

df = pd.read_csv(SAMPLE_DIR / 'macro_quarterly_sample.csv', index_col=0, parse_dates=True)
y = df['gdp_growth_qoq'].to_numpy()
print('lag1 autocorr (rough):', np.corrcoef(y[1:], y[:-1])[0,1])
```

</details>

<details><summary>Solution: HAC SE</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 04_inference_time_series_hac — HAC SE
from src import econometrics

res = econometrics.fit_ols(df, y_col='gdp_growth_qoq', x_cols=['T10Y2Y'])
res_hac = econometrics.fit_ols_hac(df, y_col='gdp_growth_qoq', x_cols=['T10Y2Y'], maxlags=2)
print('naive SE:', res.bse)
print('HAC SE  :', res_hac.bse)
```

</details>

